# Data Augmentation

- gray value augmentation
- flips, rotations, scaling.
- random crops

## Imports and Constants, etc.

In [ ]:
import datetime
import importlib
import keras
from keras.layers import (Dense, SimpleRNN, Input, Conv1D, 
                          LSTM, GRU, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D,
                          Conv3D, UpSampling3D, BatchNormalization, Concatenate, Add,
                          GaussianNoise, Dropout
                         )
from keras.models import Model
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import projd
import random
import re
import scipy
import shutil
import SimpleITK # xvertseg MetaImage files
import sys
from sklearn.model_selection import train_test_split
import uuid

import matplotlib.pyplot as plt # data viz
import seaborn as sns # data viz

import imageio # display animated volumes
from IPython.display import Image # display animated volumes

from IPython.display import SVG # visualize model
from keras.utils.vis_utils import model_to_dot # visualize model

# for importing local code
src_dir = str(Path(projd.cwd_token_dir('notebooks')) / 'src') # $PROJECT_ROOT/src
if src_dir not in sys.path:
    sys.path.append(src_dir)

import augmentation
import util
import preprocessing
import datagen
import modelutil
import xvertseg


MODEL_NAME = 'model_14'
SEED = 25 # random seed
EPOCHS = 100
BATCH_SIZE = 4
PATCH_SHAPE = (32, 32, 32)
VALIDATION_SPLIT = 0.25

DATA_DIR = Path('/data2').expanduser()
# UVMMC
NORMAL_SCANS_DIR = DATA_DIR / 'uvmmc/nifti_normals'
PROJECT_DATA_DIR = DATA_DIR / 'uvm_deep_learning_project'
PP_IMG_DIR = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed' # preprocessed scans dir
PP_MD_PATH = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed_metadata.pkl'
# xVertSeg
XVERTSEG_DIR = DATA_DIR / 'xVertSeg.v1'
PP_XVERTSEG_DIR = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed' # preprocessed scans dir
PP_XVERTSEG_MD_PATH = PROJECT_DATA_DIR / 'xVertSeg.v1' / 'preprocessed_metadata.pkl'


MODELS_DIR = PROJECT_DATA_DIR / 'models'
LOG_DIR = PROJECT_DATA_DIR / 'log'
TENSORBOARD_LOG_DIR = PROJECT_DATA_DIR / 'tensorboard' / MODEL_NAME
TMP_DIR = DATA_DIR / 'tmp'

for d in [DATA_DIR, NORMAL_SCANS_DIR, PROJECT_DATA_DIR, PP_IMG_DIR, MODELS_DIR, LOG_DIR, 
          TENSORBOARD_LOG_DIR, TMP_DIR, PP_MD_PATH.parent, PP_XVERTSEG_DIR, PP_XVERTSEG_MD_PATH.parent]:
    if not d.exists():
        d.mkdir(parents=True)
        
%matplotlib inline
sns.set()

# I love u autoreload!
%load_ext autoreload
%autoreload 2

## Gray Value Augmentation

Increase all the values in the image by a random proportion.  This lightens or darkens the image.

In [ ]:
infos = preprocessing.read_preprocessed_metadata(PP_MD_PATH)


In [ ]:
path = infos.loc[12, 'pp_path']
img = preprocessing.get_preprocessed_image(path)

In [ ]:
print('img shape:', img.shape)
util.animate_crop(img, step=10)

In [ ]:
for i in range(2):
    aug = augmentation.augment_image(img, gray_std=0.25, crop_shape=(128, 128, 128), gray_disco=True)
    print('augmented shape:', aug.shape)
    display(util.animate_crop(aug, step=10))

## Flip Augmentation

In [ ]:
for i in range(2):
    aug = augmentation.augment_image(img, flip=0.5)
    print('augmented shape:', aug.shape)
    display(util.animate_crop(aug, step=10))

## Transposition Augmentation

Transpose the axes.  Like flipping along a diagonal.

In [ ]:
for i in range(4):
    aug = augmentation.augment_image(img, crop_shape=(128,)*3, transpose=True)
    print('augmented shape:', aug.shape)
    display(util.animate_crop(aug, step=10))